In [11]:
import pandas as pd

### Sample Dataset

TDB = pd.read_excel("FoodMart.xls")

TDB.head()

,Acetominifen,Anchovies,Aspirin,Auto Magazines,Bagels,Batteries,Beer,Bologna,Candles,Canned Fruit,...,Sunglasses,Tofu,Toilet Brushes,Tools,Toothbrushes,Tuna,TV Dinner,Waffles,Wine,Yogurt
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# populate a dictionary of total purchases for each item
products = {}

for column in list(TDB):
    products[column] = sum(TDB[column])
    
# need to find a reasonable threshold (support)
import numpy as np
print("supp approx: " + str(float(np.mean([x[1] for x in products.items()])/len(TDB))))

# suggests a support threshold of ~ .033 (lowered it slightly since skewed data) or a count size of ~70
supp_threshold = 70

# get only items above threshold:
purchase_totals_above_threshold = [x for x in products.items() if x[1] > supp_threshold]
purchase_totals_above_threshold.sort(key=lambda x: x[1], reverse=True) # sorting
purchase_totals_above_threshold

###

# create new TDB with ordered max to min transactions in each row
sorted_columns = [x[0] for x in purchase_totals_above_threshold]

updated_TDB = TDB.reindex_axis(sorted_columns, axis=1, copy=True, method=None) 
# method=none ie drop columns that aren't in the new labels sorted_columns
# copy=true return a new object

updated_TDB.head()    

print("TOTAL COUNTS FOR FREQUENT 1-ITEMSETS: ")
print(purchase_totals_above_threshold)

supp approx: 0.03987306064880113
TOTAL COUNTS FOR FREQUENT 1-ITEMSETS: 
[('Cheese', 285), ('Soup', 280), ('Dried Fruit', 256), ('Cookies', 238), ('Preserves', 185), ('Wine', 185), ('Frozen Vegetables', 173), ('Canned Vegetables', 172), ('Nuts', 170), ('Milk', 164), ('Paper Wipes', 164), ('Chocolate Candy', 152), ('Chips', 145), ('Sliced Bread', 137), ('Cooking Oil', 134), ('Muffins', 132), ('Waffles', 132), ('Eggs', 128), ('Dips', 127), ('Cereal', 125), ('Popsicles', 124), ('Pizza', 122), ('Lightbulbs', 120), ('Batteries', 119), ('Coffee', 119), ('Frozen Chicken', 119), ('Deli Meats', 118), ('Juice', 113), ('Personal Hygiene', 113), ('Soda', 106), ('TV Dinner', 103), ('French Fries', 100), ('Spices', 98), ('Bologna', 97), ('Pasta', 97), ('Flavored Drinks', 96), ('Shampoo', 93), ('Jelly', 91), ('Peanut Butter', 88), ('Popcorn', 88), ('Deli Salads', 86), ('Hot Dogs', 85), ('Hard Candy', 83), ('Jam', 83), ('Pot Cleaners', 82), ('Hamburger', 81), ('Rice', 81), ('Cleaners', 80), ('Ice Cream

In [41]:
### Basic generic tree class
### Nodes have: a count, and children pointers (add_children automatically construct parent pointers)

class Node(object):
    def __init__(self, data):
        self.data = data
        self.count = 1
        self.children = []
        self.parent=[]

    def add_child(self, obj):
        obj.parent = self
        self.children.append(obj)

        
### Test Data
n = Node("A")
p = Node("B")
q = Node(7)
r = Node(8)
s = Node("C")
n.add_child(p)
n.add_child(r)
L = [p,q,r,s]

for node in L:
    if node in n.children:
        node.count += 1
    else:
        n.add_child(node)
        
for c in n.children:
    print (c.data, c.count)
    

if p.data in [y.data for y in n.children]:
    print("yes")
    
    
print( n.children[2].children)
n.children[2].add_child(q) 
print("and now")
print( n.children[2].children[0].data)


B 2
8 2
7 1
C 1
yes
[]
and now
7


In [23]:
### Build Tree
# read data from updated_TDB into a tree
root_node = Node('')
node_list = [root_node] # node list for debug purpose

import time

start_time = time.time()

for i,row in updated_TDB.iterrows():
    parent_node = root_node
    for j in range(len(row)):
        if row[j] > 0:
            # print('yes')
            name = sorted_columns[j]
            if name in [y.data for y in parent_node.children]:
                index = [y.data for y in parent_node.children].index(name)
                parent_node.children[index].count += 1
                parent_node = parent_node.children[index]
                
            else:
                x = Node(name)
                parent_node.add_child(x)
                parent_node = x
        # else:   print('no')
    print('='*30)
    
    
stop_time = time.time()

print("took " + str(stop_time-start_time))


took 1.6433470249176025


In [15]:
#[print(x.data, x.children) for x in node_list]
print([(node.data, node.count) for node in root_node.children])
    

[('Cheese', 285), ('Coffee', 16), ('Soup', 241), ('Shampoo', 10), ('Milk', 67), ('Dried Fruit', 191), ('Cooking Oil', 38), ('Waffles', 21), ('Tuna', 6), ('Peanut Butter', 7), ('Chips', 51), ('Pizza', 16), ('Paper Wipes', 68), ('Deli Meats', 16), ('Chocolate Candy', 51), ('Muffins', 31), ('Preserves', 106), ('Nuts', 62), ('Cookies', 157), ('Eggs', 25), ('Sliced Bread', 48), ('Personal Hygiene', 18), ('Popsicles', 16), ('Dips', 33), ('Lightbulbs', 18), ('Ice Cream', 6), ('Wine', 93), ('French Fries', 9), ('Canned Vegetables', 75), ('Frozen Chicken', 19), ('Hot Dogs', 9), ('Spices', 7), ('Frozen Vegetables', 77), ('Flavored Drinks', 7), ('Batteries', 20), ('Pasta', 11), ('Juice', 11), ('Cereal', 21), ('Bologna', 12), ('Cold Remedies', 6), ('Popcorn', 8), ('Soda', 10), ('Hard Candy', 8), ('Deli Salads', 5), ('TV Dinner', 13), ('Hamburger', 3), ('Donuts', 3), ('Pot Cleaners', 3), ('Plastic Utensils', 4), ('Jam', 3), ('Jelly', 6), ('Rice', 5), ('Cleaners', 1)]


In [21]:
### Could roll this into a verification function later

# Unpacking function to check counts (partially verify tree structure):
def unpack(root_node, node_list = []):
    for node in root_node.children:
        node_list.append([node.data, node.count])
        unpack(node, node_list)
    return node_list

tree = unpack(root_node)

### compress node counts for same product

summary = {}
for item in tree:
    if item[0] not in summary:
        summary[item[0]] = item[1] 
    else:
        summary[item[0]] += item[1]

all_nodes = sorted(summary.items(), key=lambda x: x[1], reverse = True) 

# all_nodes == purchase_totals_above_threshold # returns an error
# inspection shows only flags false on a few items with same purchase counts:
[ (x,y, x==y) for x,y in zip(all_nodes,purchase_totals_above_threshold) ] 

### Correct counts confirmed

[]

In [126]:
### Building Conditional FP trees

# Get leaves (bottom nodes)
def get_leafs(root_node, node_list = []):
    if root_node.children == []:
        node_list.append(root_node)
    else:
        for node in root_node.children:
            get_leafs(node, node_list)
    return node_list

# recall root_node is at apex/root of built FP tree (roll into a function)
# check root_node.children

leafs = get_leafs(root_node)
# print([x.data for x in roots])


### create tree from leaves (back traversal)

def grow_skeleton(node, tree_skeleton = []):
    tree_skeleton.append(node) 
    if node.parent == []:
        tree_skeleton.reverse()
        return tree_skeleton # so final skeleton goes from root to leaf
    else:
        grow_skeleton(node.parent, tree_skeleton)
        #tree_skeleton.reverse()
        return tree_skeleton

# ### Testing:

# X = Node("X")
# Y = Node("Y")
# Z = Node("Z")

# X.add_child(Z)

# print(X.children[0].data)
# print(Z.parent.data)
# print([x.data for x in grow_skeleton(Z)])



In [132]:
### TEST BLOCK

leafs = get_leafs(root_node)

[x.data for x in leafs]

leaf0 = leafs[0]
leaf1 = leafs[1]

print(leaf0 == leaf1, grow_skeleton(leaf0,[]) == grow_skeleton(leaf1,[]))

print([x.data for x in grow_skeleton(leaf0,[])])
print([x.data for x in grow_skeleton(leaf1,[])])

len(grow_skeleton(leaf1,[]))


False False
['', 'Cheese', 'Shampoo', 'Ice Cream']
['', 'Cheese', 'Milk', 'Hard Candy', 'Rice']


5

In [135]:
### build a conditional FP tree

leafs = get_leafs(root_node)
conditional_tree_list = []

for leaf in leafs:
    conditional_tree_list.append(grow_skeleton(leaf,[]))    

for subtree in conditional_tree_list:
    print([x.data for x in subtree])
    
### Mine conditional FP trees

['', 'Cheese', 'Shampoo', 'Ice Cream']
['', 'Cheese', 'Milk', 'Hard Candy', 'Rice']
['', 'Cheese', 'Milk', 'Rice']
['', 'Cheese', 'Milk', 'Chocolate Candy', 'Eggs', 'Popcorn', 'Pot Cleaners']
['', 'Cheese', 'Milk', 'Paper Wipes', 'Personal Hygiene', 'French Fries']
['', 'Cheese', 'Milk', 'Paper Wipes', 'Dips', 'Shampoo']
['', 'Cheese', 'Milk', 'Cooking Oil', 'Popsicles', 'Deli Meats', 'TV Dinner']
['', 'Cheese', 'Dried Fruit', 'Frozen Chicken', 'Plastic Utensils']
['', 'Cheese', 'Dried Fruit', 'Frozen Chicken', 'Jelly', 'Popcorn']
['', 'Cheese', 'Dried Fruit', 'Personal Hygiene', 'Jelly', 'Ice Cream']
['', 'Cheese', 'Dried Fruit', 'Cookies', 'Wine']
['', 'Cheese', 'Dried Fruit', 'Cookies', 'Chocolate Candy', 'Muffins', 'Lightbulbs', 'Juice', 'Spices', 'Pasta', 'Peanut Butter', 'Deli Salads', 'Plastic Utensils', 'Cold Remedies']
['', 'Cheese', 'Dried Fruit', 'Cookies', 'Paper Wipes', 'Pot Cleaners']
['', 'Cheese', 'Dried Fruit', 'Preserves', 'Frozen Chicken']
['', 'Cheese', 'Dried Fruit

['', 'Dried Fruit', 'Wine', 'Eggs', 'Bologna']
['', 'Dried Fruit', 'Wine', 'Soda', 'Spices', 'Jelly', 'Hot Dogs']
['', 'Dried Fruit', 'Wine', 'Waffles', 'Hard Candy']
['', 'Dried Fruit', 'Wine', 'Milk', 'Waffles', 'Popsicles', 'Pasta']
['', 'Dried Fruit', 'Wine', 'Muffins', 'Eggs', 'Deli Meats']
['', 'Dried Fruit', 'Wine', 'Chips', 'Cooking Oil', 'Waffles', 'Eggs', 'Bologna']
['', 'Dried Fruit', 'Wine', 'Dips']
['', 'Dried Fruit', 'Wine', 'Peanut Butter', 'Donuts']
['', 'Dried Fruit', 'Wine', 'Chocolate Candy', 'French Fries', 'Jam', 'Hamburger']
['', 'Dried Fruit', 'Wine', 'Chocolate Candy', 'Coffee', 'Jam']
['', 'Dried Fruit', 'Wine', 'Deli Meats', 'Popcorn']
['', 'Dried Fruit', 'Wine', 'Paper Wipes', 'Lightbulbs']
['', 'Dried Fruit', 'Lightbulbs', 'Donuts']
['', 'Dried Fruit', 'Lightbulbs', 'French Fries']
['', 'Dried Fruit', 'Milk', 'Dips', 'Deli Salads', 'Cold Remedies']
['', 'Dried Fruit', 'Milk', 'Chocolate Candy', 'Lightbulbs', 'TV Dinner', 'Jelly']
['', 'Dried Fruit', 'Milk', 

['', 'Soup', 'Chocolate Candy', 'Popcorn', 'Tuna']
['', 'Soup', 'Chocolate Candy', 'Waffles', 'Cleaners']
['', 'Soup', 'Chocolate Candy', 'Muffins', 'Juice', 'Donuts']
['', 'Soup', 'Chocolate Candy', 'Muffins', 'Waffles', 'TV Dinner', 'Popcorn', 'Cleaners']
['', 'Soup', 'Chocolate Candy', 'Eggs', 'Cereal', 'Plastic Utensils', 'Donuts']
['', 'Soup', 'Cooking Oil', 'Ice Cream']
['', 'Soup', 'Pizza', 'Soda', 'Spices', 'Shampoo', 'Pot Cleaners']
['', 'Soup', 'Pizza', 'Popcorn', 'Deli Salads']
['', 'Soup', 'Deli Meats']
['', 'Soup', 'Hot Dogs', 'Hard Candy', 'Cold Remedies']
['', 'Soup', 'Juice', 'Deli Salads']
['', 'Soup', 'Dips', 'TV Dinner']
['', 'Soup', 'Dips', 'Lightbulbs', 'Frozen Chicken', 'Rice']
['', 'Soup', 'Dips', 'Lightbulbs', 'Hamburger']
['', 'Soup', 'Popcorn']
['', 'Soup', 'Donuts', 'Cold Remedies']
['', 'Soup', 'French Fries', 'Cleaners', 'Donuts']
['', 'Soup', 'Muffins', 'Spices', 'Cleaners']
['', 'Soup', 'Bologna']
['', 'Soup', 'Coffee', 'Cleaners']
['', 'Soup', 'Coffee', 

['', 'Soup', 'Dried Fruit', 'Cookies', 'Jelly']
['', 'Soup', 'Dried Fruit', 'Cookies', 'Tuna']
['', 'Soup', 'Dried Fruit', 'Cookies', 'Lightbulbs', 'Hot Dogs']
['', 'Soup', 'Dried Fruit', 'Cookies', 'Preserves', 'Waffles', 'Pizza', 'Bologna', 'Pasta']
['', 'Soup', 'Dried Fruit', 'Cookies', 'Nuts', 'Muffins']
['', 'Soup', 'Dried Fruit', 'Cookies', 'Dips', 'Lightbulbs', 'Plastic Utensils']
['', 'Soup', 'Dried Fruit', 'Canned Vegetables', 'Pizza', 'TV Dinner']
['', 'Soup', 'Dried Fruit', 'Cereal', 'Deli Meats']
['', 'Soup', 'Dried Fruit', 'Cereal', 'Popcorn', 'Pot Cleaners']
['', 'Soup', 'Dried Fruit', 'Waffles', 'Spices', 'Jelly', 'Cold Remedies']
['', 'Soup', 'Dried Fruit', 'Chips', 'Pizza', 'Rice']
['', 'Soup', 'Dried Fruit', 'Chocolate Candy', 'Popsicles', 'Batteries', 'French Fries', 'Jam']
['', 'Soup', 'Dried Fruit', 'Paper Wipes', 'Lightbulbs', 'TV Dinner', 'Spices', 'Flavored Drinks', 'Cleaners']
['', 'Soup', 'Dried Fruit', 'Paper Wipes', 'Batteries']
['', 'Soup', 'Dried Fruit', '

['', 'Milk', 'Bologna', 'Jam']
['', 'Milk', 'Cereal', 'Jelly', 'Tuna']
['', 'Milk', 'Cereal', 'Deli Meats', 'Peanut Butter', 'Donuts']
['', 'Milk', 'Cereal', 'Deli Meats', 'Personal Hygiene', 'Peanut Butter', 'Donuts']
['', 'Milk', 'Personal Hygiene']
['', 'Milk', 'Ice Cream']
['', 'Milk', 'Shampoo', 'Ice Cream']
['', 'Milk', 'Dips', 'Cereal', 'Lightbulbs', 'Coffee', 'Frozen Chicken', 'Personal Hygiene', 'Spices', 'Cold Remedies']
['', 'Milk', 'Dips', 'Cereal', 'Deli Meats']
['', 'Milk', 'Dips', 'Personal Hygiene', 'Pot Cleaners']
['', 'Milk', 'Dips', 'Flavored Drinks', 'Jelly', 'Popcorn']
['', 'Milk', 'Dips', 'Peanut Butter']
['', 'Milk', 'Eggs', 'Peanut Butter']
['', 'Milk', 'Eggs', 'Personal Hygiene', 'Popcorn', 'Tuna']
['', 'Milk', 'Eggs', 'Deli Meats']
['', 'Milk', 'Eggs', 'Bologna']
['', 'Milk', 'TV Dinner', 'Peanut Butter', 'Hamburger']
['', 'Milk', 'Pizza', 'Pasta']
['', 'Milk', 'Pizza', 'Rice', 'Cleaners']
['', 'Milk', 'Rice']
['', 'Milk', 'Coffee', 'Soda', 'Bologna']
['', 'Mi

...

['', 'Chips', 'Cooking Oil', 'Frozen Chicken', 'Deli Salads']
['', 'Chips', 'Cooking Oil', 'Pizza', 'Spices']
['', 'Chips', 'Cooking Oil', 'Juice', 'Flavored Drinks', 'Hot Dogs', 'Pot Cleaners']
['', 'Chips', 'Cooking Oil', 'Personal Hygiene']
['', 'Chips', 'Plastic Utensils', 'Cold Remedies']
['', 'Chips', 'TV Dinner', 'Shampoo', 'Ice Cream', 'Donuts']
['', 'Chips', 'Popsicles', 'Flavored Drinks', 'Pot Cleaners']
['', 'Chips', 'Eggs', 'Jam']
['', 'Chips', 'Eggs', 'Pizza', 'Lightbulbs', 'French Fries']
['', 'Chips', 'Eggs', 'Popsicles']
['', 'Chips', 'Soda', 'Shampoo', 'Deli Salads', 'Hamburger', 'Ice Cream']
['', 'Chips', 'Personal Hygiene', 'Soda', 'Jelly']
['', 'Chips', 'Rice']
['', 'Chips', 'Hot Dogs']
['', 'Chips', 'Hamburger']
['', 'Chips', 'Pizza', 'Batteries', 'Juice', 'Pot Cleaners']
['', 'Chips', 'Pizza', 'Batteries', 'Juice', 'Flavored Drinks']
['', 'Chips', 'Peanut Butter']
['', 'Chips', 'Batteries', 'Rice']
['', 'Chips', 'Batteries', 'Coffee', 'Pot Cleaners', 'Hamburger', 

['', 'Paper Wipes', 'Chips', 'Personal Hygiene', 'Shampoo']
['', 'Paper Wipes', 'Chips', 'Juice']
['', 'Paper Wipes', 'Chips', 'Popsicles', 'Peanut Butter']
['', 'Paper Wipes', 'Eggs', 'Juice', 'Pasta']
['', 'Paper Wipes', 'Eggs', 'French Fries']
['', 'Paper Wipes', 'Flavored Drinks', 'Plastic Utensils']
['', 'Paper Wipes', 'Flavored Drinks', 'Pot Cleaners', 'Rice', 'Tuna']
['', 'Paper Wipes', 'French Fries']
['', 'Paper Wipes', 'Personal Hygiene', 'Popcorn']
['', 'Deli Meats', 'Ice Cream']
['', 'Deli Meats', 'TV Dinner']
['', 'Deli Meats', 'Pasta', 'Shampoo']
['', 'Deli Meats', 'Soda', 'French Fries', 'Pasta']
['', 'Deli Meats', 'Donuts']
['', 'Deli Meats', 'Juice', 'Soda']
['', 'Deli Meats', 'Juice', 'Spices']
['', 'Deli Meats', 'Hamburger', 'Plastic Utensils']
['', 'Deli Meats', 'Hot Dogs']
['', 'Chocolate Candy', 'Chips', 'Flavored Drinks', 'Peanut Butter']
['', 'Chocolate Candy', 'Chips', 'Flavored Drinks', 'Pot Cleaners']
['', 'Chocolate Candy', 'Chips', 'Lightbulbs', 'Frozen Chi

['', 'Muffins', 'Cleaners', 'Donuts']
['', 'Muffins', 'Pizza']
['', 'Muffins', 'Plastic Utensils']
['', 'Muffins', 'Jam', 'Cold Remedies']
['', 'Muffins', 'Jam', 'Tuna']
['', 'Muffins', 'Shampoo']
['', 'Muffins', 'Tuna']
['', 'Preserves', 'Canned Vegetables', 'Bologna']
['', 'Preserves', 'Canned Vegetables', 'Sliced Bread', 'Batteries']
['', 'Preserves', 'Canned Vegetables', 'Lightbulbs', 'Coffee']
['', 'Preserves', 'Canned Vegetables', 'Jam']
['', 'Preserves', 'Canned Vegetables', 'Popsicles', 'Batteries', 'Shampoo', 'Donuts']
['', 'Preserves', 'Canned Vegetables', 'Waffles', 'Jelly']
['', 'Preserves', 'Canned Vegetables', 'Milk', 'French Fries', 'Jelly']
['', 'Preserves', 'Canned Vegetables', 'Hamburger']
['', 'Preserves', 'Lightbulbs', 'Personal Hygiene', 'Soda', 'Hamburger', 'Cleaners']
['', 'Preserves', 'Deli Salads', 'Cleaners']
['', 'Preserves', 'Coffee', 'Cold Remedies']
['', 'Preserves', 'Coffee', 'Soda']
['', 'Preserves', 'Coffee', 'TV Dinner', 'Pot Cleaners']
['', 'Preserves

['', 'Cookies', 'Muffins', 'Frozen Chicken', 'Juice', 'French Fries', 'Ice Cream']
['', 'Cookies', 'Muffins', 'Soda']
['', 'Cookies', 'Muffins', 'Pasta', 'Cold Remedies']
['', 'Cookies', 'Muffins', 'Waffles', 'Jelly']
['', 'Cookies', 'Muffins', 'Hamburger']
['', 'Cookies', 'Muffins', 'Popsicles', 'Spices', 'Ice Cream']
['', 'Cookies', 'Cooking Oil', 'Pizza']
['', 'Cookies', 'Cooking Oil', 'Donuts']
['', 'Cookies', 'Cooking Oil', 'Batteries', 'Peanut Butter', 'Cleaners']
['', 'Cookies', 'Cooking Oil', 'Popsicles', 'Frozen Chicken', 'Peanut Butter', 'Hot Dogs']
['', 'Cookies', 'Cooking Oil', 'Cereal', 'Deli Meats', 'Deli Salads', 'Ice Cream', 'Cold Remedies']
['', 'Cookies', 'Cooking Oil', 'Juice', 'Jam', 'Cold Remedies']
['', 'Cookies', 'Cooking Oil', 'Spices', 'Jelly']
['', 'Cookies', 'Wine', 'Popsicles']
['', 'Cookies', 'Wine', 'Canned Vegetables', 'Nuts', 'Batteries', 'Plastic Utensils']
['', 'Cookies', 'Wine', 'Canned Vegetables', 'Bologna']
['', 'Cookies', 'Wine', 'Canned Vegetable

['', 'Cookies', 'Sliced Bread', 'Personal Hygiene', 'Soda', 'Spices', 'Bologna', 'Jelly']
['', 'Cookies', 'Sliced Bread', 'Frozen Chicken', 'Jelly']
['', 'Cookies', 'Sliced Bread', 'Dips', 'Spices', 'Deli Salads', 'Rice', 'Donuts']
['', 'Cookies', 'Sliced Bread', 'Batteries', 'Frozen Chicken', 'Soda', 'Peanut Butter']
['', 'Cookies', 'Sliced Bread', 'Coffee', 'Jam', 'Rice']
['', 'Cookies', 'Cereal', 'TV Dinner']
['', 'Cookies', 'Cereal', 'Shampoo', 'Jelly', 'Hot Dogs']
['', 'Cookies', 'Frozen Chicken', 'TV Dinner', 'Rice']
['', 'Cookies', 'Chocolate Candy']
['', 'Cookies', 'Paper Wipes', 'Eggs', 'Coffee', 'Frozen Chicken', 'Donuts']
['', 'Cookies', 'Paper Wipes', 'TV Dinner', 'Pasta', 'Shampoo']
['', 'Cookies', 'Paper Wipes', 'Sliced Bread', 'Waffles', 'French Fries']
['', 'Cookies', 'Paper Wipes', 'Chocolate Candy', 'Waffles', 'Coffee']
['', 'Cookies', 'Paper Wipes', 'Plastic Utensils']
['', 'Cookies', 'Paper Wipes', 'Soda', 'Jelly']
['', 'Cookies', 'Pizza', 'Hot Dogs']
['', 'Cookies'

In [137]:
##### cycle through conditional trees based on leafs, to create FP bases

from itertools import combinations # to get combinations

# recall supp_threshold = 70
min_support = 70

# get unique leaf names

leaf_names = list(set([leaf.data for leaf in leafs]))

for leaf in leaf_names:
    for subtree in conditional_tree_list:
        subtree_leaf_name = subtree[len(subtree) - 1].data
        if subtree_leaf_name == leaf:
            ##### Mine Freqeuent Itemsets
            conditional_FP_tree = [x for x in subtree if x.count >= min_support]
            singletons = [x for x in subtree if x.count < min_support]


Ice Cream
Rice
Rice
Pot Cleaners
French Fries
Shampoo
TV Dinner
Plastic Utensils
Popcorn
Ice Cream
Wine
Cold Remedies
Pot Cleaners
Frozen Chicken
Plastic Utensils
Chips
Shampoo
Cold Remedies
Jam
Cold Remedies
Cold Remedies
Deli Salads
Cleaners
Hamburger
Hard Candy
Donuts
Jam
Popsicles
TV Dinner
Pot Cleaners
Donuts
Hot Dogs
Jam
Plastic Utensils
Milk
Pasta
Plastic Utensils
Hot Dogs
Sliced Bread
Hot Dogs
Hard Candy
Batteries
Peanut Butter
Juice
Shampoo
Coffee
Cold Remedies
Hard Candy
Flavored Drinks
French Fries
Frozen Chicken
Tuna
Tuna
Rice
Frozen Chicken
Hard Candy
Batteries
Rice
Pot Cleaners
Rice
Rice
Donuts
Pot Cleaners
Donuts
Jelly
Muffins
Jam
Jelly
Jam
Cleaners
Rice
Ice Cream
Shampoo
Flavored Drinks
Bologna
Bologna
Hot Dogs
Batteries
Cleaners
Plastic Utensils
Hot Dogs
Pot Cleaners
Juice
Popcorn
French Fries
Deli Salads
Bologna
Canned Vegetables
Soda
Hot Dogs
Spices
Muffins
Jelly
Pot Cleaners
Donuts
TV Dinner
Popsicles
Pot Cleaners
Jelly
French Fries
Spices
Deli Salads
Spices
Deli Me

Eggs
Cold Remedies
Cold Remedies
Deli Meats
Batteries
Cold Remedies
Jelly
Pot Cleaners
Cleaners
Pot Cleaners
Flavored Drinks
Plastic Utensils
French Fries
Juice
Shampoo
Batteries
Ice Cream
Soda
Plastic Utensils
Rice
Jam
Rice
Cold Remedies
Plastic Utensils
French Fries
Hot Dogs
Pot Cleaners
Flavored Drinks
Rice
Jelly
Hot Dogs
Hamburger
Pasta
Lightbulbs
Coffee
TV Dinner
Personal Hygiene
Cold Remedies
Plastic Utensils
Deli Salads
Pizza
Jelly
Flavored Drinks
Donuts
Personal Hygiene
TV Dinner
Hamburger
Tuna
Cold Remedies
Bologna
Coffee
Soda
Juice
Peanut Butter
Rice
Ice Cream
Plastic Utensils
Cleaners
Batteries
Pizza
Tuna
Tuna
Jelly
Batteries
French Fries
Juice
Eggs
Bologna
Donuts
Hard Candy
Hamburger
Tuna
Plastic Utensils
Hard Candy
Tuna
Plastic Utensils
Deli Salads
Personal Hygiene
Frozen Chicken
Flavored Drinks
Shampoo
Plastic Utensils
Ice Cream
Deli Meats
Hard Candy
Plastic Utensils
Spices
Lightbulbs
Ice Cream
Ice Cream
Deli Salads
Hamburger
Deli Salads
Spices
Pot Cleaners
Personal Hygie

Jam
Cleaners
Batteries
Tuna
Cold Remedies
Donuts
Hamburger
Tuna
Ice Cream
Donuts
Coffee
Pasta
Flavored Drinks
Donuts
Lightbulbs
Shampoo
Flavored Drinks
Cooking Oil
Cereal
Chocolate Candy
Lightbulbs
Waffles
French Fries
Pasta
Chips
Tuna
Cleaners
Hard Candy
Cold Remedies
Popcorn
Hamburger
Jelly
Batteries
Hot Dogs
Deli Meats
Hard Candy
Jam
French Fries
Muffins
Pot Cleaners
Pot Cleaners
Cooking Oil
Pasta
Bologna
Jam
Popcorn
Plastic Utensils
TV Dinner
Flavored Drinks
Personal Hygiene
Popsicles
Hard Candy
Lightbulbs
Flavored Drinks
Hard Candy
Hamburger
Donuts
Plastic Utensils
Donuts
Cold Remedies
Popsicles
Waffles
TV Dinner
Chocolate Candy
Cold Remedies
Bologna
Soda
Donuts
French Fries
Hard Candy
Plastic Utensils
Cold Remedies
Tuna
Eggs
Cold Remedies
Plastic Utensils
Personal Hygiene
Pot Cleaners
Tuna
Deli Salads
Spices
Rice
Jelly
Shampoo
Popsicles
Hamburger
Jelly
Pizza
Popcorn
Cold Remedies
Spices
Jelly
Pot Cleaners
Donuts
Hard Candy
Hot Dogs
Cleaners
Hard Candy
Juice
Tuna
Tuna
Bologna
Pizz

Batteries
Tuna
Cold Remedies
Donuts
Hamburger
Tuna
Ice Cream
Donuts
Coffee
Pasta
Flavored Drinks
Donuts
Lightbulbs
Shampoo
Flavored Drinks
Cooking Oil
Cereal
Chocolate Candy
Lightbulbs
Waffles
French Fries
Pasta
Chips
Tuna
Cleaners
Hard Candy
Cold Remedies
Popcorn
Hamburger
Jelly
Batteries
Hot Dogs
Deli Meats
Hard Candy
Jam
French Fries
Muffins
Pot Cleaners
Pot Cleaners
Cooking Oil
Pasta
Bologna
Jam
Popcorn
Plastic Utensils
TV Dinner
Flavored Drinks
Personal Hygiene
Popsicles
Hard Candy
Lightbulbs
Flavored Drinks
Hard Candy
Hamburger
Donuts
Plastic Utensils
Donuts
Cold Remedies
Popsicles
Waffles
TV Dinner
Chocolate Candy
Cold Remedies
Bologna
Soda
Donuts
French Fries
Hard Candy
Plastic Utensils
Cold Remedies
Tuna
Eggs
Cold Remedies
Plastic Utensils
Personal Hygiene
Pot Cleaners
Tuna
Deli Salads
Spices
Rice
Jelly
Shampoo
Popsicles
Hamburger
Jelly
Pizza
Popcorn
Cold Remedies
Spices
Jelly
Pot Cleaners
Donuts
Hard Candy
Hot Dogs
Cleaners
Hard Candy
Juice
Tuna
Tuna
Bologna
Pizza
Bologna
Tun

Popcorn
Shampoo
Rice
Rice
Cold Remedies
Pot Cleaners
Donuts
Tuna
Donuts
Hamburger
Hamburger
Cleaners
Deli Salads
Hamburger
Hot Dogs
Shampoo
Ice Cream
Rice
Popcorn
Peanut Butter
Cleaners
Donuts
Pot Cleaners
Cold Remedies
Jam
Plastic Utensils
Cleaners
Cleaners
Ice Cream
Rice
Rice
Pot Cleaners
French Fries
Shampoo
TV Dinner
Plastic Utensils
Popcorn
Ice Cream
Wine
Cold Remedies
Pot Cleaners
Frozen Chicken
Plastic Utensils
Chips
Shampoo
Cold Remedies
Jam
Cold Remedies
Cold Remedies
Deli Salads
Cleaners
Hamburger
Hard Candy
Donuts
Jam
Popsicles
TV Dinner
Pot Cleaners
Donuts
Hot Dogs
Jam
Plastic Utensils
Milk
Pasta
Plastic Utensils
Hot Dogs
Sliced Bread
Hot Dogs
Hard Candy
Batteries
Peanut Butter
Juice
Shampoo
Coffee
Cold Remedies
Hard Candy
Flavored Drinks
French Fries
Frozen Chicken
Tuna
Tuna
Rice
Frozen Chicken
Hard Candy
Batteries
Rice
Pot Cleaners
Rice
Rice
Donuts
Pot Cleaners
Donuts
Jelly
Muffins
Jam
Jelly
Jam
Cleaners
Rice
Ice Cream
Shampoo
Flavored Drinks
Bologna
Bologna
Hot Dogs
Batt

...

Hot Dogs
Hard Candy
Pasta
Deli Meats
Bologna
Dips
Donuts
Hamburger
Jam
Popcorn
Lightbulbs
Donuts
French Fries
Cold Remedies
Jelly
Shampoo
Hard Candy
Eggs
Pasta
Cold Remedies
Batteries
Rice
Soda
Hard Candy
Pizza
Pot Cleaners
Hard Candy
Spices
Batteries
Hamburger
TV Dinner
Eggs
Cold Remedies
Cold Remedies
Deli Meats
Batteries
Cold Remedies
Jelly
Pot Cleaners
Cleaners
Pot Cleaners
Flavored Drinks
Plastic Utensils
French Fries
Juice
Shampoo
Batteries
Ice Cream
Soda
Plastic Utensils
Rice
Jam
Rice
Cold Remedies
Plastic Utensils
French Fries
Hot Dogs
Pot Cleaners
Flavored Drinks
Rice
Jelly
Hot Dogs
Hamburger
Pasta
Lightbulbs
Coffee
TV Dinner
Personal Hygiene
Cold Remedies
Plastic Utensils
Deli Salads
Pizza
Jelly
Flavored Drinks
Donuts
Personal Hygiene
TV Dinner
Hamburger
Tuna
Cold Remedies
Bologna
Coffee
Soda
Juice
Peanut Butter
Rice
Ice Cream
Plastic Utensils
Cleaners
Batteries
Pizza
Tuna
Tuna
Jelly
Batteries
French Fries
Juice
Eggs
Bologna
Donuts
Hard Candy
Hamburger
Tuna
Plastic Utensils
H

Tuna
Ice Cream
Tuna
Donuts
Personal Hygiene
Popcorn
Soda
Plastic Utensils
Hamburger
Tuna
Plastic Utensils
Hard Candy
Cold Remedies
Deli Salads
Hamburger
Tuna
Cold Remedies
Juice
Plastic Utensils
Tuna
Shampoo
Peanut Butter
Peanut Butter
Eggs
Tuna
Deli Meats
Deli Meats
Tuna
Cold Remedies
Cleaners
Cleaners
Soda
Deli Salads
French Fries
Flavored Drinks
Hard Candy
Flavored Drinks
Spices
Chips
Jam
Hot Dogs
Dips
Shampoo
Popcorn
Popcorn
Popcorn
Shampoo
Jam
Jam
Pot Cleaners
Tuna
Hard Candy
Juice
Popsicles
Tuna
Ice Cream
Hard Candy
Coffee
Spices
Donuts
Nuts
Popcorn
Hard Candy
Peanut Butter
Soda
Plastic Utensils
Jelly
Tuna
Hot Dogs
Pasta
Muffins
Plastic Utensils
TV Dinner
Deli Meats
Pot Cleaners
Cold Remedies
Rice
Jam
Cleaners
Batteries
Tuna
Cold Remedies
Donuts
Hamburger
Tuna
Ice Cream
Donuts
Coffee
Pasta
Flavored Drinks
Donuts
Lightbulbs
Shampoo
Flavored Drinks
Cooking Oil
Cereal
Chocolate Candy
Lightbulbs
Waffles
French Fries
Pasta
Chips
Tuna
Cleaners
Hard Candy
Cold Remedies
Popcorn
Hamburger

Donuts
Cold Remedies
Hot Dogs
Plastic Utensils
Coffee
Dips
Cold Remedies
Spices
Hamburger
Deli Salads
Shampoo
Hot Dogs
French Fries
Coffee
Plastic Utensils
Popsicles
Bologna
Hamburger
Sliced Bread
Pot Cleaners
Jam
Hard Candy
Cleaners
Popcorn
Rice
Hard Candy
Tuna
Hot Dogs
Plastic Utensils
Jam
Cleaners
Hot Dogs
Peanut Butter
Deli Salads
Bologna
Tuna
Deli Salads
Peanut Butter
Jelly
Hot Dogs
Flavored Drinks
Tuna
Jam
Rice
Hard Candy
Rice
Pot Cleaners
Plastic Utensils
Peanut Butter
Hot Dogs
French Fries
Shampoo
Pot Cleaners
Ice Cream
Pot Cleaners
Hamburger
Popcorn
Jelly
Jam
Hard Candy
Popcorn
Soda
Popcorn
Personal Hygiene
Cleaners
Ice Cream
Jelly
Donuts
Popcorn
Shampoo
Rice
Rice
Cold Remedies
Pot Cleaners
Donuts
Tuna
Donuts
Hamburger
Hamburger
Cleaners
Deli Salads
Hamburger
Hot Dogs
Shampoo
Ice Cream
Rice
Popcorn
Peanut Butter
Cleaners
Donuts
Pot Cleaners
Cold Remedies
Jam
Plastic Utensils
Cleaners
Cleaners
Ice Cream
Rice
Rice
Pot Cleaners
French Fries
Shampoo
TV Dinner
Plastic Utensils
Popc

French Fries
Coffee
Plastic Utensils
Popsicles
Bologna
Hamburger
Sliced Bread
Pot Cleaners
Jam
Hard Candy
Cleaners
Popcorn
Rice
Hard Candy
Tuna
Hot Dogs
Plastic Utensils
Jam
Cleaners
Hot Dogs
Peanut Butter
Deli Salads
Bologna
Tuna
Deli Salads
Peanut Butter
Jelly
Hot Dogs
Flavored Drinks
Tuna
Jam
Rice
Hard Candy
Rice
Pot Cleaners
Plastic Utensils
Peanut Butter
Hot Dogs
French Fries
Shampoo
Pot Cleaners
Ice Cream
Pot Cleaners
Hamburger
Popcorn
Jelly
Jam
Hard Candy
Popcorn
Soda
Popcorn
Personal Hygiene
Cleaners
Ice Cream
Jelly
Donuts
Popcorn
Shampoo
Rice
Rice
Cold Remedies
Pot Cleaners
Donuts
Tuna
Donuts
Hamburger
Hamburger
Cleaners
Deli Salads
Hamburger
Hot Dogs
Shampoo
Ice Cream
Rice
Popcorn
Peanut Butter
Cleaners
Donuts
Pot Cleaners
Cold Remedies
Jam
Plastic Utensils
Cleaners
Cleaners
Ice Cream
Rice
Rice
Pot Cleaners
French Fries
Shampoo
TV Dinner
Plastic Utensils
Popcorn
Ice Cream
Wine
Cold Remedies
Pot Cleaners
Frozen Chicken
Plastic Utensils
Chips
Shampoo
Cold Remedies
Jam
Cold Reme

Peanut Butter
TV Dinner
Deli Meats
Popcorn
Hot Dogs
Donuts
Donuts
Cold Remedies
Cold Remedies
Personal Hygiene
Bologna
Lightbulbs
Dips
Waffles
Ice Cream
Pasta
Pasta
Deli Meats
Hard Candy
Popcorn
Shampoo
Ice Cream
Bologna
Batteries
Cleaners
Personal Hygiene
Rice
Donuts
Hard Candy
Cold Remedies
Pizza
Popcorn
Hard Candy
Ice Cream
French Fries
Cereal
Tuna
Jam
Donuts
Popcorn
Soda
Shampoo
Pot Cleaners
Spices
Cleaners
Cleaners
Hamburger
Ice Cream
Ice Cream
Donuts
Tuna
Lightbulbs
Muffins
Tuna
Pizza
French Fries
Juice
Jam
Hamburger
Hot Dogs
Jam
Hamburger
Donuts
Rice
Hamburger
Spices
TV Dinner
Hamburger
Tuna
Spices
Rice
Hot Dogs
Frozen Chicken
Juice
Peanut Butter
Plastic Utensils
Cleaners
Eggs
Plastic Utensils
Plastic Utensils
Pot Cleaners
Cold Remedies
Batteries
Ice Cream
Flavored Drinks
TV Dinner
Popcorn
Coffee
Peanut Butter
Cleaners
Hot Dogs
Pot Cleaners
Hamburger
Spices
Batteries
Shampoo
Pot Cleaners
Cereal
Dips
Pasta
Ice Cream
Jelly
Ice Cream
Soda
Cold Remedies
Jelly
Hamburger
Ice Cream
Piz

In [124]:
def build_fp_tree(data, supp):

    # scan data and create ordered list of single itemsets
    products = {}

    for column in list(data):
        products[column] = sum(data[column])
    
    purchase_totals_all_products = data.items()
    
    # get only items above threshold:
    purchase_totals_above_threshold = [x for x in purchase_totals_all_products if x[1] > supp]
    purchase_totals_above_threshold.sort(key=lambda x: x[1], reverse=True) # sorting
    
    # create new TDB with ordered max to min transactions in each row
    
    return purchase_totals_above_threshold
    
###